In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Using TensorFlow backend.


In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = InceptionV3(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(3931, 2048)

In [8]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [9]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 .....

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.3s


[CV] C=1000.0 ........................................................
[CV] ........................... C=0.01, score=0.917303, total=   5.6s
[CV] C=1000.0 ........................................................
[CV] ........................... C=0.01, score=0.942748, total=   5.8s
[CV] C=1000.0 ........................................................
[CV] ............................ C=1.0, score=0.923761, total=  12.9s
[CV] ............................ C=0.1, score=0.935197, total=  13.3s
[CV] C=1000.0 ........................................................
[CV] C=10000.0 .......................................................
[CV] ........................... C=10.0, score=0.905852, total=  12.2s
[CV] C=10000.0 .......................................................
[CV] ........................... C=10.0, score=0.933842, total=  13.1s
[CV] ........................... C=10.0, score=0.923761, total=  13.4s
[CV] C=10000.0 .......................................................
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   16.3s


[CV] ......................... C=1000.0, score=0.926209, total=  13.3s
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.932570, total=  13.8s
[CV] ........................... C=10.0, score=0.924936, total=  18.5s
[CV] C=100000.0 ......................................................
[CV] C=100000.0 ......................................................
[CV] ............................ C=0.1, score=0.909669, total=  19.4s
[CV] C=1000000.0 .....................................................
[CV] ............................ C=1.0, score=0.928753, total=  19.1s
[CV] C=1000000.0 .....................................................
[CV] ............................ C=1.0, score=0.905852, total=  19.2s
[CV] C=1000000.0 .....................................................
[CV] ............................ C=0.1, score=0.933842, total=  20.2s
[CV] .......................... C=100.0, score=0.923761, total=  18.9s
[CV] C

...

[CV] ...................... C=1000000.0, score=0.926209, total=  30.0s
[CV] ..................... C=10000000.0, score=0.907125, total=  27.3s
[CV] ...................... C=1000000.0, score=0.932570, total=  30.5s
[CV] .................. C=10000000000.0, score=0.935115, total=  17.1s
[CV] .................. C=10000000000.0, score=0.924936, total=  18.3s
[CV] ..................... C=10000000.0, score=0.932570, total=  29.5s
[CV] ..................... C=10000000.0, score=0.926209, total=  30.6s


[Parallel(n_jobs=-1)]: Done  60 out of  65 | elapsed:   52.7s remaining:    4.4s


[CV] .................... C=100000000.0, score=0.921220, total=  29.8s
[CV] ..................... C=10000000.0, score=0.941476, total=  31.4s
[CV] .................... C=100000000.0, score=0.926209, total=  31.0s
[CV] .................. C=10000000000.0, score=0.905852, total=  13.7s
[CV] .................. C=10000000000.0, score=0.941476, total=  30.1s


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:  1.2min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03,   1.00000e+04,   1.00000e+05,
         1.00000e+06,   1.00000e+07,   1.00000e+08,   1.00000e+09,
         1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [10]:
top_model.cv_results_

{'mean_fit_time': array([  4.82212038,  17.81856217,  17.04645524,  15.27233896,
         18.73820214,  16.10239902,  13.55059361,  16.24071274,
         25.89710684,  29.32637939,  22.82463861,  16.7822896 ,  19.31347322]),
 'mean_score_time': array([ 0.07772279,  0.07785358,  0.06096859,  0.07615004,  0.06456203,
         0.02995377,  0.09645638,  0.04070663,  0.08427105,  0.09045439,
         0.06276498,  0.06675601,  0.03318415]),
 'mean_test_score': array([ 0.94250827,  0.93284152,  0.92622742,  0.92571865,  0.92571865,
         0.92571865,  0.92571865,  0.92622742,  0.92571865,  0.92571865,
         0.92622742,  0.92622742,  0.92597303]),
 'mean_train_score': array([ 0.98778948,  0.99904606,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         0.99993641,  0.99993641,  1.        ]),
 'param_C': masked_array(data = [0.01 0.10000000000000001 1.0 10.0 100.0 1000.0 10000.0 100000.0 1000000.0
  10000000.0 100000000.0 

In [11]:
top_model.grid_scores_

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.94251, std: 0.01372, params: {'C': 0.01},
 mean: 0.93284, std: 0.01219, params: {'C': 0.10000000000000001},
 mean: 0.92623, std: 0.01171, params: {'C': 1.0},
 mean: 0.92572, std: 0.01162, params: {'C': 10.0},
 mean: 0.92572, std: 0.01122, params: {'C': 100.0},
 mean: 0.92572, std: 0.01150, params: {'C': 1000.0},
 mean: 0.92572, std: 0.01150, params: {'C': 10000.0},
 mean: 0.92623, std: 0.01184, params: {'C': 100000.0},
 mean: 0.92572, std: 0.01150, params: {'C': 1000000.0},
 mean: 0.92572, std: 0.01150, params: {'C': 10000000.0},
 mean: 0.92623, std: 0.01184, params: {'C': 100000000.0},
 mean: 0.92623, std: 0.01184, params: {'C': 1000000000.0},
 mean: 0.92597, std: 0.01218, params: {'C': 10000000000.0}]

In [12]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9425
